# PRODUCTION phase: About this notebook
- Purpose: Creates 1 AKS webservice, to serve the model as an ONLINE endpoint
    - `AKS ONLINE Webservice:` Fetches the best trained model, Deployes that on an `AKS cluster`, always up and running, ready to be pinged and return results (via REST / Swagger, or Python SDK)

## DETAILS - about this notebook and the 2 pipelines, generated            
- 1) `Initiates ESMLProject` and sets `active model` and `active date folder`:
- 2) `DEPLOY & SERVER: Fetched the BEST MODEL, and deploys on AKS`
- 3) `Smoke testing: Fetches some data and calls the webservice` (smoke testing purpose - see that it works...)
    - Gets test data
    - Calls webservice, which both returns data via REST call, and ESML optionally also saves the returned result to datalake 

# Login / Switch DEV_TEST_PROD environment (1-timer)

import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLProject

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../settings/model/active/active_scoring_in_folder.json',
p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
#auth = InteractiveLoginAuthentication(force=True, tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)

# 1) `Initiates ESMLProject` and sets `active model` and `active date folder`:

In [ ]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")
from esmlrt.interfaces.iESMLController import IESMLController
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../settings/model/active/active_scoring_in_folder.json',
p.active_model = 11
p.inference_mode = False
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.verbose_logging = False

## 2a) `DEPLOY`: Option A - Let ESML find BEST Model ( and its environment, scoring script) 

In [ ]:
inference_config, model, best_run = IESMLController.get_best_model_inference_config(p.ws, p.model_folder_name, p.ModelAlias)
service,api_uri, kv_aks_api_secret= p.deploy_model_as_private_aks_online_endpoint(model,inference_config,overwrite_endpoint=True)

## 2b) `DEPLOY`: Option B - Inject YOUR selection of model and run, any model, override "ESML best model logic"

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
from azureml.pipeline.core import PipelineRun
from azureml.train.automl.run import AutoMLRun

def get_best_model_inference_config(model_name,model_version, run_id = None):
    print(" - model_name {} | version {} | run_id: {}".format(model_name,model_version,run_id))
    model = Model(workspace=p.ws,name=model_name, version=model_version)
    experiment = Experiment(p.ws,p.experiment_name )
    best_run = None
    if(run_id is not None):
        main_run = PipelineRun(experiment=experiment, run_id=run_id)
        best_run = main_run
    inference_config, model, best_run = IESMLController.get_best_model_inference_config(p.ws, p.model_folder_name,p.ModelAlias,scoring_script_folder_local=None, current_model=model,run_id_tag=run_id, best_run = best_run)
    return inference_config, model, best_run

#### Option B (Model A): ...Deploy ANY model (not only the best promoted model)

In [ ]:
model_name = 'your_model_folder_name' # '11_diabetes_model_reg' - Find a model name in Azure ML Studio/Model register or lake_settings.json
model_version = 1 # Find a model name in Azure ML Studio/Model register
run_id = 'todo_c70-3ef4-470c-9f55-92b33318c8ad'# '9360ac70-3ef4-470c-9f55-92b33318c8ad' # Main pipeline run - Find a model name in Azure ML Studio/Model register

inference_config, model, best_run = get_best_model_inference_config(model_name,model_version,run_id)
service,api_uri, kv_aks_api_secret= p.deploy_model_as_private_aks_online_endpoint(model,inference_config,overwrite_endpoint=True)

# 3) `Smoke testing:` TEST ENDPOINT - Score with some test data

### Get testdata, and score it

In [ ]:
p.connect_to_lake()
X_test, y_test, tags = p.get_gold_validate_Xy()
caller_id = "10965d9c-40ca-4e47-9723-5a608a32a0e4" # Pass an optional tracking ID for the request, parquet file will then have this name

df = p.call_webservice(p.ws, X_test,caller_id) # Saves to datalake also
#df = p.call_webservice(ws=p.ws, pandas_X_test=X_test.iloc[:1],user_id=caller_id,firstRowOnly=True,save_2_lake_also=False) # If only 1 row, and not saving result to datalake
pd.set_option('display.max_colwidth', None)
df.head()

## END) `SUMMARY - what did the notebook do:DEPLOY & SERVE: Fetched the BEST MODEL, and deploys on AKS`
- ESML saves `API_key in Azure keyvault automatically`
- ESML auto-config solves 4 common 'errors/things': `correct compute name` and `valid replicas, valid agents, valid auto scaling` in 
    -  AKS_SETTINGS: [aks_config_dev.json](../settings/project_specific/model/dev_test_prod_override/online/aks_config_dev.json)
- Tip 1: You can adjust the number of replicas, and different CPU/memory configuration, or using a different compute target.

### Logging: enable in [aks_config_dev.json](../settings/project_specific/model/dev_test_prod_override/online/aks_config_dev.json)
You can enable logging with LogAnalytics, with the flags

    `enable_app_insights:true`
    `collect_model_data:true`

### AKS_SETTINGS
- Here you can edit AKS settings (performance for DEV, TEST, PROD environments) under PROJECT specific MODEL settings and ONLINE = AKS
    -  Link: [aks_config_dev.json](../settings/project_specific/model/dev_test_prod_override/online/aks_config_dev.json)
- Note: 
    - Q: Why is `docker_bridge_cidr` a PUBLIC IP? Isn't this a PRIVATE AKS cluster?
    - A: Yes, it is PRIVATE. We don’t use the docker bridge for pod communication, but as Docker is configured as part of the Kubernetes setup, this docker bridge it also gets created as well, so in order to avoid that it picks random unknown CIDR that could collide with any of your existent subnets, we give the option to change it and set it a known range. So the indication for docker bridge is to define any CIDR that doesn’t to Azure, and doesn’t collide with any other subnet. 
        - Read more: [learn.microsoft.com](https://learn.microsoft.com/en-us/answers/questions/199786/how-to-update-docker-bridge-cidr-for-aks-to-a-diff.html)

# END